# Example notebook for XAI algorithms quantitive evaluation

This tutorial is based on [Quantus notebook](https://github.com/understandable-machine-intelligence-lab/Quantus/blob/main/tutorials/Tutorial_ImageNet_Quantification_with_Quantus.ipynb) with guideline how to get started using [Quantus](https://github.com/understandable-machine-intelligence-lab/Quantus) library.

In this tutorial, You will see how You can use explainable algorithms to study pre-trained model decision and quantify those algorithms using metrics provided by Quantus library. We will take a pre-trained model, sample images, run several explainable methods and quantify them for further analysis.

### Setup 

Install missing libraries.

In [ ]:
!pip install opencv-python seaborn quantus cachetools

#### Imports

First we have to import all necessary libraries.

In [ ]:
# import necessary libraries
import os
import torch
from torch.utils.data import DataLoader
import torchvision

from foxai.context_manager import FoXaiExplainer, ExplainerWithParams, CVClassificationExplainers
from foxai.visualizer import mean_channels_visualization

import quantus
import numpy as np
from foxai.explainer.computer_vision.algorithm.integrated_gradients import IntegratedGradientsCVExplainer
from foxai.explainer.computer_vision.algorithm.input_x_gradient import InputXGradientCVExplainer
from foxai.explainer.computer_vision.algorithm.gradient_shap import GradientSHAPCVExplainer
from foxai.explainer.computer_vision.algorithm.guided_backprop import GuidedBackpropCVExplainer
from foxai.explainer.computer_vision.algorithm.gradcam import LayerGradCAMCVExplainer
from foxai.explainer.computer_vision.algorithm.gradcam import GuidedGradCAMCVExplainer
from functools import partial

import cv2
import matplotlib.pyplot as plt

from torchvision.transforms._presets import ImageClassification
from typing import Tuple, List, Dict, Callable, Optional
import seaborn as sns
import pandas as pd
from time import time

# Plotting specifics.
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

Configure `CUDA_LAUNCH_BLOCKING=1` to prevent issues with `CUDA` while running GPU-accelerated computations in notebook.

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

#### Define custom functions

Define function that will load model, list of labels and transformation function of a desired model. Currently we support, in this notebook, only a few models: `VGG11`, `ResNet50`, `ViT`, `MobileNetV3` and `YOLOv5`. You can easilly add new models from `torchvision` model zoo and even define Your own model.

In [ ]:
def load_model(
    model_name: str,
) -> Tuple[torch.nn.Module, List[str], ImageClassification]:
    """Load model, label list and transformation function used in data preprocessing.

    Args:
        model_name: Model name. Recognized models are: `vgg11`, `resent50`, `vit` and
            `mobilenetv3`.

    Raises:
        ValueError: raised if provided model name that is not supported.

    Returns:
        Tuple of model, list of labels and transformation function.
    """
    # normalize model name to match recognized models
    model_name_normalized: str = model_name.lower().strip()
    if model_name_normalized == "vgg11":
        weights = torchvision.models.VGG11_Weights.IMAGENET1K_V1

        # load model from torchvision model zoo
        model = torchvision.models.vgg11(weights=weights)

        # get class names
        categories = weights.meta["categories"]
        transform = weights.transforms()
    elif model_name_normalized == "vit":
        weights = torchvision.models.ViT_B_16_Weights.IMAGENET1K_V1

        # load model from torchvision model zoo
        model = torchvision.models.vit_b_16(weights=weights)

        # get class names
        categories = weights.meta["categories"]
        transform = weights.transforms()
    elif model_name_normalized == "resnet50":
        weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V1

        # load model from torchvision model zoo
        model = torchvision.models.resnet50(weights=weights)

        # get class names
        categories = weights.meta["categories"]
        transform = weights.transforms()
    elif model_name_normalized == "mobilenetv3":
        weights = torchvision.models.MobileNet_V3_Small_Weights.IMAGENET1K_V1

        # load model from torchvision model zoo
        model = torchvision.models.mobilenet_v3_small(weights=weights)

        # get class names
        categories = weights.meta["categories"]
        transform = weights.transforms()
    else:
        raise ValueError(f"Unrecognized model name: {model_name}")

    return model, categories, transform

### Configuration

Cell below contains configuration of this notebook. We have defined max number of samples to be saved in artifact directory, path to `ImageNet-Mini` dataset downloaded from [Kaggle](https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000), name of the model, batch_size and device to be used.

In [ ]:
batch_size: int = 2
max_samples_explained: int = 10
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model_name: str = "mobilenetv3"

# define directory where explanation artifacts will be stored
artifact_dir: str = f"artifacts/{model_name}/"

# `data_dir` variable contains path to dataset downloaded from https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000.
# You have to register in Kaggle to be able to download this dataset.
data_dir: str = "/home/user/Downloads/imagenet-mini"


### Loading the model

Load specified model, put it in evaluation mode, place it on specified device, download and preprocess `ImageNet-Mini` dataset. Trasformation function is used to match training dataset preprocessing steps.

In [ ]:
# load model, classes and transformation function
model, categories, transform = load_model(model_name=model_name)

# put model in evaluation mode
model.eval()

# place model on specified device (CPU or GPU)
model.to(device)

# get last layer to explain
layer = [module for module in model.modules() if isinstance(module, torch.nn.Conv2d)][-1]

# load test dataset - ImageNet-Mini downloaded from Kaggle: https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000
imagenet_val = torchvision.datasets.ImageFolder(root=f"{data_dir}/val", transform=transform)
val_dataloader = DataLoader(imagenet_val, batch_size=batch_size)

In [ ]:
# instruct notebook to display figures inline
%matplotlib inline

In [ ]:
# Source code: https://matplotlib.org/stable/gallery/specialty_plots/radar_chart.html.

def radar_factory(num_vars, frame='circle'):
    """Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle' | 'polygon'}
        Shape of frame surrounding axes.
    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarAxes(PolarAxes):

        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default."""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default."""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels, angles=None):
            self.set_thetagrids(angles=np.degrees(theta), labels=labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            """ Draw. If frame is polygon, make gridlines polygon-shaped."""
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)


        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)

                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta


# Plotting configs.
sns.set(font_scale=1.8)
plt.style.use('seaborn-white')
plt.rcParams['ytick.labelleft'] = True
plt.rcParams['xtick.labelbottom'] = True

def plot_spyder_graph(
    df_normalised_rank: pd.DataFrame,
    xai_methods: List[str],
    metric_weight_dict: Dict[str, float],
    include_titles: bool = True,
    include_legend: bool = True,
) -> None:
    # Make spyder graph!
    data = [df_normalised_rank.columns.values, (df_normalised_rank.to_numpy())]
    theta = radar_factory(len(data[0]), frame='polygon')
    _ = data.pop(0)

    fig, ax = plt.subplots(figsize=(11, 11), subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(top=0.85, bottom=0.05)
    for d, method in zip(data[0], xai_methods):
        _ = ax.plot(
            theta,
            d,
            label=method,
            linewidth=5.0
        )
        ax.fill(theta, d, alpha=0.15)

    # Set lables.
    if include_titles:
        ax.set_varlabels(labels=list(metric_weight_dict.keys()))
    else:
        ax.set_varlabels(labels=[]) 

    # Set a title.
    ax.set_title("Summary of Explainer Quantification",  position=(0.5, 1.1), ha='center', fontsize=15)

    # Put a legend to the right of the current axis.
    if include_legend:
        ax.legend(loc='upper left', bbox_to_anchor=(1, 0.5))

    plt.tight_layout()
    plt.show()

In [ ]:
def foxai_explainer_wrapper(
    explainer, model, inputs, targets, *args, **kwargs
) -> np.array:
    """Wrapper aorund captum's Saliency implementation."""
    torch.cuda.empty_cache()

    # Set model in evaluate mode.
    model.to(kwargs.get("device", None))
    model.eval()

    if not isinstance(inputs, torch.Tensor):
        inputs = torch.as_tensor(inputs)

    if not isinstance(targets, torch.Tensor):
        targets = (
            torch.as_tensor(targets).long()
        )  # inputs = inputs.reshape(-1, 3, 224, 224)

    assert (
        len(np.shape(inputs)) == 4
    ), "Inputs should be shaped (nr_samples, nr_channels, img_size, img_size) e.g., (1, 3, 224, 224)."

    explanation = (
        explainer()
        .calculate_features(model, inputs, targets)
        .sum(axis=1)
        .detach()
        .cpu()
        .numpy()
    )

    torch.cuda.empty_cache()

    if not kwargs.get("normalise", None):
        explanation = quantus.normalise_func.normalise_by_negative(explanation)

    if isinstance(explanation, torch.Tensor):
        if explanation.requires_grad:
            return explanation.detach().cpu().detach().numpy()
        return explanation.detach().cpu().numpy()

    return explanation

integrated_gradients_explainer = partial(foxai_explainer_wrapper, explainer=IntegratedGradientsCVExplainer)
input_x_gradients_explainer = partial(foxai_explainer_wrapper, explainer=InputXGradientCVExplainer)
gradient_shap_explainer = partial(foxai_explainer_wrapper, explainer=GradientSHAPCVExplainer)
guided_backprop_explainer = partial(foxai_explainer_wrapper, explainer=GuidedBackpropCVExplainer)
layer_gradcam_explainer = partial(foxai_explainer_wrapper, explainer=LayerGradCAMCVExplainer)
guided_gradcam_explainer = partial(foxai_explainer_wrapper, explainer=GuidedGradCAMCVExplainer)

explainer_wrapper = {
    "IntegratedGradients": integrated_gradients_explainer,
    "InputXGradient": input_x_gradients_explainer,
    "GradientSHAP": gradient_shap_explainer,
    "GuidedBackpropagation": guided_backprop_explainer,
    "LayerGradCAM": layer_gradcam_explainer,
    "GuidedGradCAM": guided_gradcam_explainer,
}

In [ ]:
def get_batch_data(
    dataloader: DataLoader,
    device: torch.device,
    max_batch_no: int,
) -> Tuple[List[torch.Tensor], List[torch.Tensor]]:
    """Get list of batches of data sample and labels.

    Args:
        dataloader: Dataloader to iterate.
        device: torch.Device to place batches to.
        max_batch_no: Number of batches.

    Returns:
        Tuple of lists of batches of data sample and labels.
    """
    x_batch_list: List[torch.Tensor] = []
    y_batch_list: List[torch.Tensor] = []
    for index, batch in enumerate(dataloader):
        sample_batch, label_batch = batch
        sample_batch.to(device)
        label_batch.to(device)

        x_batch_list.append(sample_batch)
        y_batch_list.append(label_batch)

        if index > max_batch_no:
            break

    return x_batch_list, y_batch_list

def compute_metrics_for_explainers(
    x_batch_list: List[torch.Tensor],
    y_batch_list: List[torch.Tensor],
    explainer_wrapper: Dict[str, Callable],
    model: torch.nn.Module,
    layer: torch.nn.Module,
    device: torch.device,
) -> Dict[str, Dict[str, List[np.ndarray]]]:
    explainer_attributes: Dict[str, Dict[str, List[np.ndarray]]] = {key: [] for key in explainer_wrapper.keys()}
    for sample_batch, label_batch in zip(x_batch_list, y_batch_list):
        for explainer_name, explainer in explainer_wrapper.items():
            start = time()
            
            attributes = explainer(
                model=model,
                inputs=sample_batch,
                targets=label_batch,
                **{
                    "device": device,
                    "layer": layer,    
                },
            )
            explainer_attributes[explainer_name].append(attributes)

            stop = time()
            print(f"Explainer name: {explainer_name}, time: {round(stop - start, 2)} seconds")


    for explainer_key in explainer_attributes.keys():
        explainer_attributes[explainer_key] = np.vstack(explainer_attributes[explainer_key])

    return explainer_attributes

x_batch_list, y_batch_list = get_batch_data(
    dataloader=val_dataloader,
    device=device,
    max_batch_no=2,
)
x_batch = np.vstack(x_batch_list)
y_batch = np.hstack(y_batch_list)
explainer_attributes = compute_metrics_for_explainers(
    x_batch_list=x_batch_list,
    y_batch_list=y_batch_list,
    explainer_wrapper=explainer_wrapper,
    model=model,
    layer=layer,
    device=device,
)

In [ ]:
def visualize_attributes(
    index: int,
    explanations: Dict[str, List[np.ndarray]],
    max_height: int = 224,
    max_width: int = 224,
) -> None:
    """Visualize attributes from set of explainers in one plot.

    Args:
        index: Index of sample to visualize.
        explanations: Dictionary of attributes.
        max_height: Maximum height of image. Defaults to 224.
        max_width: Maximum width of image. Defaults to 224.
    """
    _, axes = plt.subplots(nrows=1, ncols=1+len(explanations), figsize=(15, 8))
    axes[0].imshow(
        np.moveaxis(
            quantus.normalise_func.denormalise(
                x_batch[index],
                mean=np.array([0.485, 0.456, 0.406]),
                std=np.array([0.229, 0.224, 0.225])
            ),
            0,
            -1
        ),
        vmin=0.0,
        vmax=1.0
    )
    axes[0].set_title(f"ImageNet class {y_batch[index].item()}", rotation=60)
    axes[0].axis("off")
    for i, (k, v) in enumerate(explanations.items()):
        if explanations[k][index].shape[0] < max_height:
            image = cv2.resize(explanations[k][index], None, fx=max_width, fy=max_height, interpolation=cv2.INTER_NEAREST)
        else:
            image = explanations[k][index]
        axes[i+1].imshow(
            quantus.normalise_func.normalise_by_negative(image),
            cmap="seismic",
            vmin=-1.0,
            vmax=1.0
        )
        axes[i+1].set_title(f"{k}", rotation=60)
        axes[i+1].axis("off")


visualize_attributes(index=1, explanations=explainer_attributes)

## Metrics

In the cell below we will define metrics that will be used for quantifying explainable algorithms. Those metrics, according to authors of Quantus library, are sensitive to different configurations and need to be carefully parametrized. To better understand how those parameters influence metric consider reading original papers describing metrics and study implementation code of Quantus library.

Library itself provides guidance about different parameters that may influence results in logs. It also provides original publication details.

In [ ]:
# Define XAI methods and metrics.
xai_methods = list(explainer_attributes.keys())
metrics = {
    "EffectiveComplexity": quantus.EffectiveComplexity(
        abs=True,
        normalise=False,
        aggregate_func=np.mean,
        return_aggregate=True,
        disable_warnings=False,
    ),
    "EffectiveComplexityNormalised": quantus.EffectiveComplexity(
        abs=True,
        normalise=True,
        aggregate_func=np.mean,
        return_aggregate=True,
        disable_warnings=False,
    ),
    "Sparseness": quantus.Sparseness(
        abs=True,
        normalise=False,
        aggregate_func=np.mean,
        return_aggregate=True,
        disable_warnings=False,
    ),
    "SparsenessNormalised": quantus.Sparseness(
        abs=True,
        normalise=True,
        aggregate_func=np.mean,
        return_aggregate=True,
        disable_warnings=False,
    ),
    "Complexity": quantus.Complexity(
        abs=True,
        normalise=False,
    ),
    "ComplexityNormalised": quantus.Complexity(
        abs=True,
        normalise=True,
    ),
}

In [ ]:
def compute_metrics(
    xai_methods: Dict[str, Callable],
    x_batch: torch.Tensor,
    y_batch: torch.Tensor,
    explainer_attributes: Dict[str, List[np.ndarray]],
    metrics: Dict[str, Callable],
) -> Dict[str, Dict[str, List[np.ndarray]]]:
    """Calculate metric score for each explainer.

    Args:
        xai_methods: Dictionary of explainer name and function.
        x_batch: Image batch tensor.
        y_batch: Label batch tensor.
        explainer_attributes: Dictionary of attributes for each explainer.
        metrics: Dictionary of metric name and function.

    Returns:
        Dictionary with name of metric and score for each explainer algorithm.
    """
    # run quantification analysis
    results = {method : {} for method in xai_methods}

    if isinstance(x_batch, torch.Tensor):
        x_batch = x_batch.detach().cpu().numpy()
    if isinstance(y_batch, torch.Tensor):
        y_batch = y_batch.detach().cpu().numpy()

    for method in explainer_attributes.keys():
        for metric, metric_func in metrics.items():
            start = time()
            torch.cuda.empty_cache()

            # Get scores and append results.
            scores = metric_func(
                model=model,
                x_batch=x_batch,
                y_batch=y_batch,
                a_batch=explainer_attributes[method],
                s_batch=None,
                device=device,
                explain_func=explainer_wrapper[method],
                explain_func_kwargs={
                    "device": device,
                },
            )
            results[method][metric] = scores
            stop = time()
            print(f"Evaluating {metric} of {method} method, time: {round(stop - start, 2)} seconds")

            # Empty cache.
            torch.cuda.empty_cache()

    return results

results = compute_metrics(
    xai_methods=xai_methods,
    x_batch=x_batch,
    y_batch=y_batch,
    explainer_attributes=explainer_attributes,
    metrics=metrics,
)

In [ ]:
for key, metric_dict in results.items():
    print(f"Algorithm '{key}'")
    for metric_name, metric_value in metric_dict.items():
        print(f"\t'{metric_name}': {round(metric_value[0], 2)}")

In [ ]:
def calculate_metric_ranks(
        metrics: Dict[str, Callable],
        xai_methods: Dict[str, List[np.ndarray]],
        results: Dict[str, Dict[str, List[np.ndarray]]],
        lower_is_better_metric_names: List[str],
) -> pd.DataFrame:
    # Postprocessing of scores: to get how the different explanation methods rank across criteria.
    results_agg = {}
    for method in xai_methods:
        results_agg[method] = {}
        for metric, _ in metrics.items():
            results_agg[method][metric] = np.mean(results[method][metric])

    df = pd.DataFrame.from_dict(results_agg)
    df = df.T.abs()

    # Take inverse ranking for EffectiveComplexity, EffectiveComplexityNormalized, Sparsity and SparsityNormalized since lower is better.
    df_normalised = df.loc[:].apply(lambda x: x / x.max())

    # apply normalisation for `lower is beter` metrics
    for metric_name in lower_is_better_metric_names:
        df_normalised[metric_name] = df[metric_name].min() / df[metric_name].values

    df_normalised_rank = df_normalised.rank()

    # replace NaN with zeros
    for column in df.columns:
        df_normalised_rank[column] = df_normalised_rank[column].replace(np.nan, -1)

    return df_normalised_rank

df_normalised_rank = calculate_metric_ranks(
    metrics=metrics,
    xai_methods=xai_methods,
    results=results,
    lower_is_better_metric_names=[
        "EffectiveComplexityNormalised",
        "EffectiveComplexity",
        "Sparseness",
        "SparsenessNormalised",
    ],
)
df_normalised_rank

Aggregate and sort XAI algorithm score based on weighted XAI metrics partial scores. You can use this approach to automatically select the most prominent XAI algorithm for given dataset and model. This approach is only demonstration of possible use case scenario. At the moment we don't have one robust approach to select XAI algorithm without human intervention. When selecting XAI metrics you need to know them in depth to use and interpret them correctly.

In [ ]:
def get_explainers_score(
    df_normalised_rank: pd.DataFrame,
    metric_weight_dict: Optional[Dict[str, float]] = None,
) -> Dict[str, float]:
    """Return dictionary of explainers with aggregated weighted metric score in descending order.

    Args:
        metric_weight_dict: Dictionary with weights for every metric.
        df_normalised_rank: DataFrame with ranks assigned to every explainer.

    Returns:
        dictionary of explainers with aggregated weighted metric score.
    """
    # use equally weighted metrics in this example
    if metric_weight_dict is None:
        metric_weight_dict = {key: 1.0 for key in metrics.keys()}

    # calculate final score
    final_score = {}
    for row in df_normalised_rank.iterrows():
        weighted_score = sum([weight * row[1].loc[key] for key, weight in metric_weight_dict.items()])
        final_score[row[0]] = weighted_score

    sorted_explaienr_list = sorted(final_score.items(), key=lambda x:x[1], reverse=True)
    return sorted_explaienr_list

metric_weight_dict = {key: 1.0 for key in metrics.keys()}
best_explainer = get_explainers_score(
    df_normalised_rank=df_normalised_rank,
    metric_weight_dict=metric_weight_dict,
)

print("Best explainers based on weighted metric score (higher is better)")
for explainer in best_explainer:
    print(f"\tExplainer_name: '{explainer[0]}', score: {explainer[1]}")

## Metrics parametrization

On the spyder graph you can compare different XAI algorithms based on provided metrics. In this example we have used 3 metrics with and without normalization to show you that it's important to take parametrization into account as the results may differ significantly. For `Sparseness` and `Complexity` normalisation is not a problem but for `Effective Complexity` it is.

In [ ]:
plot_spyder_graph(
    df_normalised_rank=df_normalised_rank,
    xai_methods=xai_methods,
    metric_weight_dict=metric_weight_dict,
)

## More metrics

In [ ]:
explainer_wrapper = {
    "IntegratedGradients": integrated_gradients_explainer,
    "GradientSHAP": gradient_shap_explainer,
}

# Define XAI methods and metrics.
xai_methods = list(explainer_wrapper.keys())
metrics = {
    "NonSensitivity": quantus.NonSensitivity(
        n_samples=100,    # number of samples to generate
        features_in_step=512, # number of features permuted in each step; the higher number the faster each step is computed
        aggregate_func=np.mean,
        return_aggregate=True,
        disable_warnings=False,
        display_progressbar=True,   # important to see progress bar and estimate time of computations
    ),
    "Sparseness": quantus.Sparseness(
        abs=True,
        normalise=False,
        aggregate_func=np.mean,
        return_aggregate=True,
        disable_warnings=False,
    ),
    "Complexity": quantus.Complexity(
        abs=True,
        normalise=False,
    ),
}

In [ ]:
explainer_attributes = compute_metrics_for_explainers(
    x_batch_list=x_batch_list,
    y_batch_list=y_batch_list,
    explainer_wrapper=explainer_wrapper,
    model=model,
    layer=layer,
    device=device,
)

In [ ]:
results = compute_metrics(
    xai_methods=xai_methods,
    x_batch=x_batch,
    y_batch=y_batch,
    explainer_attributes=explainer_attributes,
    metrics=metrics,
)

df_normalised_rank = calculate_metric_ranks(
    metrics=metrics,
    xai_methods=xai_methods,
    results=results,
    lower_is_better_metric_names=[
        "Sparseness",
    ],
)
df_normalised_rank

In [ ]:
metric_weight_dict = {key: 1.0 for key in metrics.keys()}
best_explainer = get_explainers_score(
    df_normalised_rank=df_normalised_rank,
    metric_weight_dict=metric_weight_dict,
)

print("Best explainers based on weighted metric score (higher is better)")
for explainer in best_explainer:
    print(f"\tExplainer_name: '{explainer[0]}', score: {explainer[1]}")

plot_spyder_graph(
    df_normalised_rank=df_normalised_rank,
    xai_methods=xai_methods,
    metric_weight_dict=metric_weight_dict,
)